In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from gensim.models import Word2Vec
import re

# 학습 데이터 준비
csv_file_path1 = r"C:\Users\권현주\Downloads\한국언론진흥재단_뉴스빅데이터_메타데이터_가짜뉴스_20201231.csv"
csv_file_path2 = r"C:\Users\권현주\Desktop\asdf.csv"
df2 = pd.read_csv(csv_file_path2)
df1 = pd.read_csv(csv_file_path1)

df1['참과 거짓'] = 0
df2['참과 거짓'] = 1
df2.rename(columns={'특성추출(가중치순 상위 50개)':'특성추출'},inplace = True)
df = pd.concat([df1, df2], ignore_index=True)
df = df.drop(['통합 분류1', '통합 분류2', '통합 분류3', '사건_사고 분류1', '사건_사고 분류2', '사건_사고 분류3', '개체명(인물)', '개체명(지역)', '개체명(기업기관)'], axis=1)
df_train = df
df_train['본문'] = df_train['본문'].astype(str)


# 텍스트 전처리
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df_train['본문'])
sequences = tokenizer.texts_to_sequences(df_train['본문'])
max_length = max(len(sequence) for sequence in sequences)  # 입력 시퀀스 최대 길이
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')

# 입력 데이터와 레이블을 Tensor로 변환
train_data = tf.convert_to_tensor(padded_sequences)
train_labels = tf.convert_to_tensor(df_train['참과 거짓'])


# 단어 집합 크기 설정
vocab_size = len(tokenizer.word_index) + 1  # 단어 집합 크기 계산


# Word2Vec 모델 훈련
embedding_dim = 100  # Word2Vec 임베딩 벡터 차원 수
sentences = [tokenizer.sequences_to_texts([sequence.numpy()])[0].split() for sequence in train_data]
w2v_model = Word2Vec(sentences=sentences, vector_size=embedding_dim, window=5, min_count=1, workers=4)

# LSTM 모델 생성
hidden_units = 64  # LSTM 레이어의 유닛 수
num_classes = 2  # 분류할 클래스(레이블) 수
num_epochs = 100  # 학습 반복 횟수
batch_size = 32  # 미니배치 크기

embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, idx in tokenizer.word_index.items():
    if word in w2v_model.wv:
        embedding_matrix[idx] = w2v_model.wv[word]

# LSTM 모델 생성
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length,
                              weights=[embedding_matrix], trainable=False),
    tf.keras.layers.LSTM(units=hidden_units),
    tf.keras.layers.Dense(units=num_classes, activation='softmax')
])


# 모델 컴파일
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 모델 학습
model.fit(train_data, train_labels, epochs=num_epochs, batch_size=batch_size)

csv_file_path2 = r"C:\Users\권현주\Downloads\한국언론진흥재단_뉴스빅데이터_메타데이터_청와대_19971231.csv"
td = pd.read_csv(csv_file_path2, encoding='cp949')
# 테스트 데이터 준비
test_data = td

# 테스트 데이터 전처리
test_sequences = tokenizer.texts_to_sequences(test_data)
test_sequences = pad_sequences(test_sequences, maxlen=max_length, padding='post')

# 입력 데이터를 Tensor로 변환
test_data = tf.convert_to_tensor(test_sequences)

# 모델 예측
predictions = model.predict(test_data)
predicted_labels = np.argmax(predictions, axis=1)

# 예측 결과 출력
fake_news_count = 0
for i, predicted_label in enumerate(predicted_labels):
    if predicted_label == 0:
            fake_news_count += 1
print(f"전체 테스트 데이터 중 가짜뉴스 개수: {fake_news_count}")

Epoch 1/100
1297/1297 [==============================] - 23s 16ms/step - loss: 0.1939 - accuracy: 0.9179
Epoch 2/100
1297/1297 [==============================] - 20s 16ms/step - loss: 0.1320 - accuracy: 0.9461
Epoch 3/100
1297/1297 [==============================] - 22s 17ms/step - loss: 0.1155 - accuracy: 0.9516
Epoch 4/100
1297/1297 [==============================] - 22s 17ms/step - loss: 0.1089 - accuracy: 0.9543
Epoch 5/100
1297/1297 [==============================] - 21s 16ms/step - loss: 0.1001 - accuracy: 0.9585
Epoch 6/100
1297/1297 [==============================] - 21s 16ms/step - loss: 0.0946 - accuracy: 0.9601
Epoch 7/100
1297/1297 [==============================] - 21s 16ms/step - loss: 0.0872 - accuracy: 0.9635
Epoch 8/100
1297/1297 [==============================] - 19s 15ms/step - loss: 0.0830 - accuracy: 0.9645
Epoch 9/100
1297/1297 [==============================] - 19s 15ms/step - loss: 0.0808 - accuracy: 0.9654
Epoch 10/100
1297/1297 [==============================]

In [ ]:
csv_file_path2 = r"C:\Users\권현주\Downloads\한국언론진흥재단_뉴스빅데이터_메타데이터_가짜뉴스_20201231 (1).csv"
td = pd.read_csv(csv_file_path2, encoding='cp949')
# 테스트 데이터 준비
test_data = td

# 테스트 데이터 전처리
test_sequences = tokenizer.texts_to_sequences(test_data)
test_sequences = pad_sequences(test_sequences, maxlen=max_length, padding='post')

# 입력 데이터를 Tensor로 변환
test_data = tf.convert_to_tensor(test_sequences)

# 모델 예측
predictions = model.predict(test_data)
predicted_labels = np.argmax(predictions, axis=1)

# 예측 결과 출력
fake_news_count = 0
for i, predicted_label in enumerate(predicted_labels):
    if predicted_label == 0:
            fake_news_count += 1
print(f"전체 테스트 데이터 중 가짜뉴스 개수: {fake_news_count}")

In [ ]:
# 모델 예측
predictions = model.predict(test_data)
predicted_labels = np.argmax(predictions, axis=1)

# 예측 결과 출력
fake_news_count = 0
for i, predicted_label in enumerate(predicted_labels):
    if predicted_label == 0:
        fake_news_count += 1
print(f"전체 테스트 데이터 중 가짜뉴스 개수: {fake_news_count}")

# 가짜뉴스로 예측된 뉴스의 인덱스 출력
fake_news_indices = []
for i, predicted_label in enumerate(predicted_labels):
    if predicted_label == 0:
        fake_news_indices.append(i)

print("가짜뉴스로 예측된 뉴스의 인덱스:")
for idx in fake_news_indices:
    print(idx)

In [37]:
df.head()

,주소,일자,언론사,기고자,제목,키워드,특성추출,본문,원본주소,참과 거짓,인물,위치,기관,특성추출(가중치순 상위 50개)
0,http://www.bigkinds.or.kr/news/newsDetailView....,2020-01-01,조선일보,워싱턴/김신영 기자,"진짜뉴스는 비싸다, 때론 목숨까지 건다","진짜뉴스,목숨,조선일보,진실,수호자들,저널리즘,도전,민주주의,위협,가짜,전쟁,거짓,...","뉴지엄,박물관,추모관,미국,로스앤젤레스타임스,전시물,캐논,비가트,패런솔드,기자들,워...",미국 워싱턴DC 국회의사당 부근 뉴스 박물관인 뉴지엄(Newseum) 3층엔 세상을...,https://news.chosun.com/site/data/html_dir/202...,1,NaN,NaN,NaN,NaN
1,http://www.bigkinds.or.kr/news/newsDetailView....,2020-01-01,매일경제,김규리,라임사태 피해 투자자들 대규모 법정대응 예고…`불완전 판매` 여부 핵심,"라임,사태,피해,투자자,법정,대응,예고,불완전,판매,여부,핵심,투자처,라임,자산,운...","라임자산운용,투자자,광화,iig,투자자들,헤지펀드,미국,무역금융펀드,등록취소,판매사...",라임자산운용 무역금융펀드 투자처인 미국 헤지펀드 운용사가 등록취소 제재를 받으면서 ...,http://news.mk.co.kr/newsRead.php?no=1773&year...,1,NaN,NaN,NaN,NaN
2,http://www.bigkinds.or.kr/news/newsDetailView....,2020-01-01,아주경제,전환욱,​유성엽 “갈등·분노·혐오 사라지고 화합·통합의 새해 열리길”,"유성엽,갈등,분노,혐오,화합,통합,새해,한국,통일,추진,세상,유성엽,대안,창당준비위...","위원장,유성엽,창당준비위원장,신년사,광주시당,중앙당,대한민국,강대국,일본,성원,광주...",유성엽 대안신당 창당준비위원장이 1일 신년사에서 ..“우리 사회의 건강한 발전을 가...,http://www.ajunews.com/view/20200101160438501,1,NaN,NaN,NaN,NaN
3,http://www.bigkinds.or.kr/news/newsDetailView....,2020-01-01,SBS,김도균 기자,AI가 만든 '진짜 같은 가짜'…선거판에 악용될까 '우려',"AI,진짜,가짜,선거판,악용,앵커,인공지능,환영,악용,걱정,목소리,총선,미국,대선,...","딥페이크,인공지능,미국,중국,김도균,오바마,선거판,화면제공,이흥규,원형희,카이스트,...","<앵커> .. .. .. .. .. ..인공지능이 환영받기만 하는 건 아니지요, 기...",https://news.sbs.co.kr/news/endPage.do?news_id...,1,NaN,NaN,NaN,NaN
4,http://www.bigkinds.or.kr/news/newsDetailView....,2020-01-01,서울경제,노현섭 기자,‘벵가지’ 악몽에 ‘이라크 美대사관 공격’ 강경 대응 나선 트럼프,"벵가지,악몽,이라크,대사관,공격,강경,대응,트럼프,도널드,트럼프,미국,대통령,이라크...","벵가지,미국,이라크,대사관,행정부,클린턴,폼페이오,리비아,트윗,오바마,바그다드,워싱...",도널드 트럼프 미국 대통령이 이라크의 친이란 시아파 민병대 카티이브-헤즈볼라를 폭격...,http://www.sedaily.com/NewsView/1YXH7V9DCA,1,NaN,NaN,NaN,NaN
